In [ ]:
from tqdm.notebook import tqdm
import torch
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
from scipy.special import expit

import sys
sys.path.append('..')

from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet,weights
from isplutils import utils

In [ ]:
!pip install torch

In [ ]:

net_model = 'EfficientNetAutoAttB4'
train_db = 'FFPP'

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 244
frames_per_video = 60

In [ ]:
model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))

In [ ]:
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

In [ ]:
facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn,facedet=facedet)

In [ ]:
test_dir = '/test_videos/'
filenames=os.listdir(test_dir)
test_video_files = [test_dir + x for x in filenames]
res = []
for video in tqdm(test_video_files):
    vid_faces = face_extractor.process_video(video)
    faces_t = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in vid_faces if len(frame['faces'])] )
    with torch.no_grad():
        faces_pred = torch.sigmoid(net(faces_t.to(device))).cpu().numpy().flatten()
    res.append(faces_pred[0])

In [ ]:
df_test=pd.read_csv('EfficientNetB4AttModel.csv')
df_test['label']=res
df_test['filename']=filenames
df_test.to_csv('EfficientNetB4AttModel.csv',index=False)